### Parameter setting
#### * Adjustments are required according to the trained model *

In [1]:
from keras import models
from keras import layers
from keras import optimizers
import numpy as np
import copy
import itertools 
import pickle

In [2]:
get_model_directory = lambda series, cut: f"models/{series}_series/{cut}_cut/"
get_model_name = lambda number:  f"m_{number}.h5"
simplexDir = lambda series, cut, number: f"simplexes/{series}_series/{cut}_cut/simp_{number}/"

# filList = range(1,65)

### Construct simplex

In [3]:
def get_relevance(model, outputSize = 1, input_layer=True, weights = None):
    if input_layer:
        layers = model.layers
    else:
        layers = model.layers[1:]
        
    if weights == None:
        weights = [layer.get_weights()[0] for layer in layers]
    sizes = [len(weight) for weight in weights] + [outputSize]

    offset = 0
    size = sum(sizes)
    relevance = np.identity(size)

    for layer_num in range(len(sizes) - 1, 0, -1):
        old_offset = offset
        offset += sizes[layer_num]

        weight = weights[layer_num - 1]
        weightPlus = weight * (weight > 0)
        for j in range(0, sizes[layer_num]):
            normalizeFactor = 0
            for i in range(sizes[layer_num - 1]):
                normalizeFactor += weightPlus[i][j]
            for i in range(sizes[layer_num - 1]):
                x, y = i + offset, j + old_offset
                if weightPlus[i][j] != 0:
                    relevance[x][y] = weightPlus[i][j] / normalizeFactor
    return np.array(relevance) 

In [4]:
def make_model(input_size, parameters):
    model = models.Sequential()

    model.add(layers.Dense(parameters[0], activation='relu', input_shape=(input_size,)))
    for i in range(1, len(parameters)-1):
        model.add(layers.Dense(parameters[i], activation="relu"))
    model.add(layers.Dense(parameters[-1], activation='sigmoid'))
    
    model.compile(optimizer=optimizers.Adam(), 
              loss='binary_crossentropy', 
              metrics=["accuracy"]
              )
    
    return model

In [5]:
def comb(sequence):
    result = []
    for L in range(1, len(sequence)+1):
        for subset in itertools.combinations(sequence, L):
            result.append(list(subset))
    return result

In [6]:
def getSimplex(matrix, pointSequence, threshold):
    matrixSize = len(matrix)
    
    relevance = 1.0
    result = []
    #startPointからのRelevanceを計算する
    startPoint = pointSequence[0]
    for pointNumber in pointSequence:
        relevance = relevance * matrix[startPoint][pointNumber]
        startPoint = pointNumber
    #relevanceがthreshold以上だったらここまでの経路を追加する

    if relevance >= threshold:
        for e in comb(pointSequence):
            result.append(e)
        #最後の要素からの連結要素について再帰的にチェックする
        lastPoint = pointSequence[-1]
        for i in range(matrixSize):
            if matrix[lastPoint][i] > 0 and i != lastPoint:
                tempPointSequence = copy.deepcopy(pointSequence)
                tempPointSequence.append(i)
                #再帰呼び出し
                temp = getSimplex(matrix, tempPointSequence, threshold)
                #結果をresultに追加
                for e in temp:
                    for ee in comb(e):
                        result.append(ee)
    return list(map(list, set(map(tuple,result))))

In [7]:
import os
from tqdm import tqdm 
r = list(reversed(np.logspace(-7, 0, base=10, num=64)))

def registerSimplexOutput(filList, series, cut, id, name=None):
    if name == None:
        model = models.load_model(get_model_directory(series, cut) + get_model_name(id))
    else: 
        model = models.load_model(name)
    matrix = get_relevance(model, input_layer = True)
    matrixSize = len(matrix)
    
    for fil in tqdm(filList):
        number = r[fil - 1]
        if name == None:
            filename =  simplexDir(series, cut, id) + "Simplex" + str(fil)
        else:
            filename = "random_simp" + name + "/Simplex" + str(fil)
        
        saveSimplex = []
        for startPoint in range(0, matrixSize):
            simplex = getSimplex(matrix, [startPoint], number)
            saveSimplex.extend(simplex)
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        saveFile = open(filename, 'wb')
        pickle.dump(saveSimplex, saveFile)
        saveFile.close

In [8]:
log_cuts = [1, 
            # 2, 
            # 3, 
            # 5, 
            # 10, 
            # 20, 
            # 40, 
            # 60, 
            100, 
            300]
tries = 15
series = [22]

In [9]:
import datetime 

def log_preamble(log_file_name):
    global tries, epochs, series
    time_stamp = datetime.datetime.now()

    with open(log_file_name, "a") as log_file:
        log_file.write(f"\n\nExecuted on time is {datetime.datetime.now()}\n")
        log_file.write(f"Tries: {tries}, series = {series}\n")

    return time_stamp

def log_final(log_file_name, time_stamp):
    time_stamp_new = datetime.datetime.now()
    with open(log_file_name, "a") as log_file:
        log_file.write(f"Finished successfully at {time_stamp_new}\n")
        log_file.write(f"Total time = {(time_stamp_new - time_stamp).total_seconds()}\n")

In [10]:
log_file_name = "simplex_log.txt"

t = log_preamble(log_file_name)

for s in series:
    for cut in log_cuts:
        for x in range(tries):
            registerSimplexOutput(filList, s, cut, x)

log_final(log_file_name, t)

100%|██████████| 64/64 [00:07<00:00,  8.82it/s]
